In [1]:
import sys
import numpy as np
from versioned_hdf5.subchunk_map import as_subchunk_map

try:
    from versioned_hdf5.staged_changes import (
        ChangesPlan,
        GetItemPlan,
        ResizePlan,
        SetItemPlan,
        np_hsize_t,
    )
except ImportError:
    # master branch
    ChangesPlan = None
    GetItemPlan = None
    ResizePlan = None
    SetItemPlan = None
    np_hsize_t = np.uint64

np.random.seed(0)

print(sys.modules.get("versioned_hdf5.staged_changes"))
print(sys.modules["versioned_hdf5.subchunk_map"])

<module 'versioned_hdf5.staged_changes' from '/home/crusaderky/github/versioned-hdf5/versioned_hdf5/staged_changes.cpython-312-x86_64-linux-gnu.so'>
<module 'versioned_hdf5.subchunk_map' from '/home/crusaderky/github/versioned-hdf5/versioned_hdf5/subchunk_map.cpython-312-x86_64-linux-gnu.so'>


# GetItemPlan benchmarks

In [2]:
def bench_getitem_plan(chunk_shape, perc):
    if chunk_shape == "square":
        chunk_size = (1000, 1000)
        nchunks = (100, 100)
    elif chunk_shape == "flat":
        chunk_size = (10, 100_000)
        nchunks = (10_000, 1)
    else:
        raise ValueError(chunk_shape)

    shape = (100_000, 100_000)
    assert all(n * c == s for n, c, s in zip(nchunks, chunk_size, shape))

    idx = (slice(int(shape[0] * perc)), slice(int(shape[1] * perc)))

    # Brand new dense array
    slab_indices = np.ones(nchunks, dtype=np_hsize_t)
    slab_offsets = np.arange(
        0,
        slab_indices.size * chunk_size[0],
        chunk_size[0],
        dtype=np_hsize_t,
    ).reshape(slab_indices.shape)

    args = (idx, shape, chunk_size, slab_indices, slab_offsets)

    print(f"{chunk_shape} {perc*100:.0f}%")
    if GetItemPlan is not None:
        print(GetItemPlan(*args).head)
        %timeit GetItemPlan(*args)

    print("as_subchunk_map")
    %timeit list(as_subchunk_map(idx=idx, shape=shape, chunk_size=chunk_size))

    print()

Performance is impacted by the number of modified chunks.


All benchmarks run on a dataset 80 GB in size (assuming double or int64 data), 10,000 chunks, 8 MB per chunk and plan the execution of reading the whole dataset item (a[:, :]).

#### Legend
Shape of the chunks:
- **square**: 1000x1000 points (8MB) square chunks, 100 chunks per axis
- **flat**: a single chunk of 8MB spanning the columns back to back. Typical e.g after a conversion from Pandas or PyArrow.

In [3]:
bench_getitem_plan("square", 1)
bench_getitem_plan("flat", 1)
bench_getitem_plan("square", 0.05)
bench_getitem_plan("flat", 0.05)

square 100%
GetItemPlan<output_shape=(100000, 100000), output_view=[:, :], 10000 slice transfers among 1 slab pairs>
574 μs ± 1.92 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
as_subchunk_map
16.3 ms ± 78.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)

flat 100%
GetItemPlan<output_shape=(100000, 100000), output_view=[:, :], 10000 slice transfers among 1 slab pairs>
1.55 ms ± 8.68 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
as_subchunk_map
19.3 ms ± 182 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)

square 5%
GetItemPlan<output_shape=(5000, 5000), output_view=[:, :], 25 slice transfers among 1 slab pairs>
188 μs ± 3.6 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
as_subchunk_map
112 μs ± 614 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

flat 5%
GetItemPlan<output_shape=(5000, 5000), output_view=[:, :], 500 slice transfers among 1 slab pairs>
214 μs ± 389 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops 

# More plans benchmarks
The benchmarks below mirror those in wrapper_benchmark.ipynb

In [4]:
shape = (20_000, 20_000)  # 3 GiB
chunk_size = (250, 250)  # 488 kiB
nchunks = tuple(s // c + (s % c > 0) for s, c in zip(shape, chunk_size))
# Simulate newly created dense versioned_hdf5 dataset
slab_indices = np.ones(nchunks, dtype=np_hsize_t)
slab_offsets = np.arange(
    0,
    np.prod(nchunks) * chunk_size[0],
    chunk_size[0],
    dtype=np_hsize_t,
).reshape(slab_indices.shape)

In [5]:
# setitem of whole chunks not in memory
idx = slice(chunk_size[0], None, None)
args = (idx, shape, chunk_size, slab_indices, slab_offsets, 2, 1)
if SetItemPlan is not None:
    splan = SetItemPlan(*args)
    print(splan.head)
    %timeit SetItemPlan(*args)
print("as_subchunk_map")
%timeit list(as_subchunk_map(idx=idx, shape=shape, chunk_size=chunk_size))

SetItemPlan<value_shape=(19750, 20000), value_view=[:, :], append 1 empty slabs, 6320 slice transfers among 1 slab pairs, drop 0 slabs>
436 μs ± 576 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
as_subchunk_map
10.4 ms ± 107 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
# setitem of parts of chunks (target all chunks)
idx = tuple(slice(None, None, c) for c in chunk_size)
args = (idx, shape, chunk_size, slab_indices, slab_offsets, 2, 1)
if SetItemPlan is not None:
    splan = SetItemPlan(*args)
    print(splan.head)
    %timeit SetItemPlan(*args)
print("as_subchunk_map")
%timeit list(as_subchunk_map(idx=idx, shape=shape, chunk_size=chunk_size))

SetItemPlan<value_shape=(80, 80), value_view=[:, :], append 1 empty slabs, 6480 slice transfers among 2 slab pairs, drop 1 slabs>
511 μs ± 785 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
as_subchunk_map
10.2 ms ± 57.5 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
# setitem of parts of chunks (target every other chunk)
idx = (slice(None, None, chunk_size[0] * 2), slice(None, None, chunk_size[1]))
args = (idx, shape, chunk_size, slab_indices, slab_offsets, 2, 1)
if SetItemPlan is not None:
    splan = SetItemPlan(*args)
    print(splan.head)
    %timeit SetItemPlan(*args)
print("as_subchunk_map")
%timeit list(as_subchunk_map(idx=idx, shape=shape, chunk_size=chunk_size))

SetItemPlan<value_shape=(40, 80), value_view=[:, :], append 0 empty slabs, 3200 slice transfers among 1 slab pairs, drop 0 slabs>
406 μs ± 9.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
as_subchunk_map
5.12 ms ± 66.8 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
# setitem of parts of chunks (target every other chunk on inner axis)
idx = (slice(None, None, chunk_size[0]), slice(None, None, chunk_size[1] * 2))
args = (idx, shape, chunk_size, slab_indices, slab_offsets, 2, 1)
if SetItemPlan is not None:
    splan = SetItemPlan(*args)
    print(splan.head)
    %timeit SetItemPlan(*args)
print("as_subchunk_map")
%timeit list(as_subchunk_map(idx=idx, shape=shape, chunk_size=chunk_size))

SetItemPlan<value_shape=(80, 40), value_view=[:, :], append 0 empty slabs, 3200 slice transfers among 1 slab pairs, drop 0 slabs>
396 μs ± 3.43 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
as_subchunk_map
5.17 ms ± 52 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
# setitem of parts of chunks (checkers pattern)
idx = (slice(None, None, chunk_size[0] * 2), slice(None, None, chunk_size[1] * 2))
args = (idx, shape, chunk_size, slab_indices, slab_offsets, 2, 1)
if SetItemPlan is not None:
    splan = SetItemPlan(*args)
    print(splan.head)
    %timeit SetItemPlan(*args)
print("as_subchunk_map")
%timeit list(as_subchunk_map(idx=idx, shape=shape, chunk_size=chunk_size))

SetItemPlan<value_shape=(40, 40), value_view=[:, :], append 0 empty slabs, 1600 slice transfers among 1 slab pairs, drop 0 slabs>
340 μs ± 5.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
as_subchunk_map
2.63 ms ± 4.42 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
# commit a fully modified array
if SetItemPlan is not None:
    splan = SetItemPlan((), shape, chunk_size, slab_indices, slab_offsets, 2, 1)
    args = (shape, chunk_size, splan.slab_indices, splan.slab_offsets)
    cplan = ChangesPlan(*args)
    print(cplan.head)
    %timeit ChangesPlan(*args)

ChangesPlan<6400 chunks>
1.5 ms ± 1.82 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
# getitem after a single chunk has been changed
idx = ()
if SetItemPlan is not None:
    splan = SetItemPlan((0, 0), shape, chunk_size, slab_indices, slab_offsets, 2, 1)
    args = (idx, shape, chunk_size, splan.slab_indices, splan.slab_offsets)
    gplan = GetItemPlan(*args)
    print(gplan.head)
    %timeit GetItemPlan(*args)
print("as_subchunk_map")
%timeit list(as_subchunk_map(idx=idx, shape=shape, chunk_size=chunk_size))

GetItemPlan<output_shape=(20000, 20000), output_view=[:, :], 6400 slice transfers among 1 slab pairs>
324 μs ± 2.15 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
as_subchunk_map
10.3 ms ± 39.4 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
# resize to add 1 chunk
old_shape = (shape[0] - 2, shape[1])
new_shape = (shape[0] - 1, shape[1])
args = (old_shape, new_shape, chunk_size, slab_indices, slab_offsets, 2, 1)
if ResizePlan is not None:
    rplan = ResizePlan(*args)
    print(rplan.head)
    %timeit ResizePlan(*args)
print("as_subchunk_map")
%timeit list(as_subchunk_map(idx=(), shape=new_shape, chunk_size=chunk_size))

ResizePlan<append 0 empty slabs, 80 slice transfers among 1 slab pairs, drop 0 slabs>
306 μs ± 503 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
as_subchunk_map
10.2 ms ± 62 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
